# DSN 2020 AI BOOTCAMP QUALIFICATION HACKATHON

## Waris Sanusi; AI+ University of Ibadan

### DSN/AIPlus/2019/0408

Kowope Mart is a Nigerian-based retail company with a vision to provide quality goods, education 
and automobile services to its customers at affordable price and reduce if not eradicate charges
on card payments and increase customer satisfaction with credit rewards that can be used within 
the Mall. This hackathon aims to detect customers who are likely to default the credit rewards 
and loans by the company. 

## Importing Libraries & Dataset

 This solution is a blend of blend of 3 models. (Catboost, LgihtGBM & Random Forest Classifier)

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, auc
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold

In [2]:
train=pd.read_csv('Train.csv')
test=pd.read_csv('Test.csv')
SampleSub=pd.read_csv('SampleSubmission.csv')

## Data Pre-processing

In [3]:
train.shape, test.shape

((56000, 52), (24000, 51))

In [4]:
train.head(3)

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,NaN,0.0,charge,349.80573,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 52 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Applicant_ID    56000 non-null  object 
 1   form_field1     53471 non-null  float64
 2   form_field2     52156 non-null  float64
 3   form_field3     55645 non-null  float64
 4   form_field4     55645 non-null  float64
 5   form_field5     55645 non-null  float64
 6   form_field6     42640 non-null  float64
 7   form_field7     50837 non-null  float64
 8   form_field8     42640 non-null  float64
 9   form_field9     47992 non-null  float64
 10  form_field10    55645 non-null  float64
 11  form_field11    24579 non-null  float64
 12  form_field12    46105 non-null  float64
 13  form_field13    50111 non-null  float64
 14  form_field14    56000 non-null  int64  
 15  form_field15    33525 non-null  float64
 16  form_field16    42964 non-null  float64
 17  form_field17    44849 non-null 

In [6]:
train.isnull().sum()

Applicant_ID          0
form_field1        2529
form_field2        3844
form_field3         355
form_field4         355
form_field5         355
form_field6       13360
form_field7        5163
form_field8       13360
form_field9        8008
form_field10        355
form_field11      31421
form_field12       9895
form_field13       5889
form_field14          0
form_field15      22475
form_field16      13036
form_field17      11151
form_field18      10402
form_field19          4
form_field20        355
form_field21      15854
form_field22      20400
form_field23      28123
form_field24      13297
form_field25       5450
form_field26       7438
form_field27       9299
form_field28        355
form_field29        355
form_field30      25509
form_field31      39408
form_field32       5450
form_field33       1256
form_field34        355
form_field35      23148
form_field36       1995
form_field37       5450
form_field38        355
form_field39       4211
form_field40      43729
form_field41    

In [7]:
train['form_field47'].unique()

array(['charge', 'lending'], dtype=object)

In [8]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

train['form_field47']=le.fit_transform(train['form_field47'])
test['form_field47']=le.fit_transform(test['form_field47'])

In [10]:
train.fillna(-9999999,inplace=True)
test.fillna(-9999999,inplace=True)

In [11]:
train=train.drop(['Applicant_ID'],axis=1)

In [12]:
test=test.drop(['Applicant_ID'],axis=1)

### Splitting our train dataset into features(x) and target (y) for local validation

In [13]:
x=np.array(train.drop(['default_status'],axis=1))
y=np.array(train['default_status'])

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=1)

In [15]:
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((42000, 50), (14000, 50), (42000,), (14000,))

Feature Engineering was not conducted as it tends to reduce the accuracy of the model

All models were trained on CPU

##  Model 1- Random Forest Classifier

In [68]:
estimator=RandomForestClassifier(n_estimators=400, criterion="entropy", min_samples_split=10)

In [69]:
estimator.fit(x_train,y_train)

RandomForestClassifier(criterion='entropy', min_samples_split=10,
                       n_estimators=400)

In [70]:
val_prediction=estimator.predict_proba(x_test)[:,1]

In [71]:
auc_score=roc_auc_score(y_test,val_prediction)
auc_score

0.8427360909223807

In [72]:
y_pred=estimator.predict_proba(test)[:,1]
y_pred

array([0.37194701, 0.3431519 , 0.39980647, ..., 0.33234029, 0.58690363,
       0.23742231])

In [73]:
rfc_sub = SampleSub.copy()

In [74]:
rfc_sub['default_status']=y_pred

In [75]:
rfc_sub.to_csv('RFC_SUB.csv',index=False)

## Model 2- Catboost Classifier

10 Stratified KFold Catboost training and prediction on test dataset

In [33]:
totcb_pred=[]

fold = StratifiedKFold(n_splits=10, shuffle=False, random_state=42)

for train_index, test_index in fold.split(x,y):
    
    x_train,x_test=x[train_index], x[test_index]
    y_train,y_test=y[train_index], y[test_index]
    
    estimator_1 = CatBoostClassifier(n_estimators=5000,eval_metric='AUC',random_seed= 10, 
                                   use_best_model=True)
    
    estimator_1.fit(x_train,y_train, eval_set=[(x_train,y_train), (x_test,y_test)], 
            verbose=True,early_stopping_rounds=300)
                           
    prediction_1=estimator_1.predict_proba(x_test)[:,1]
    
    pred1 = estimator_1.predict_proba(test)[:,1]
    
    totcb_pred.append(pred1)

Learning rate set to 0.041434
0:	test: 0.7952470	test1: 0.7913263	best: 0.7913263 (0)	total: 356ms	remaining: 29m 39s
1:	test: 0.8124863	test1: 0.8047852	best: 0.8047852 (1)	total: 811ms	remaining: 33m 46s
2:	test: 0.8153081	test1: 0.8069297	best: 0.8069297 (2)	total: 1.29s	remaining: 35m 43s
3:	test: 0.8168834	test1: 0.8089260	best: 0.8089260 (3)	total: 1.49s	remaining: 30m 56s
4:	test: 0.8192858	test1: 0.8097142	best: 0.8097142 (4)	total: 1.83s	remaining: 30m 24s
5:	test: 0.8194481	test1: 0.8099740	best: 0.8099740 (5)	total: 2.02s	remaining: 27m 58s
6:	test: 0.8207521	test1: 0.8107787	best: 0.8107787 (6)	total: 2.45s	remaining: 29m 7s
7:	test: 0.8211181	test1: 0.8104633	best: 0.8107787 (6)	total: 2.68s	remaining: 27m 55s
8:	test: 0.8214959	test1: 0.8105081	best: 0.8107787 (6)	total: 3.03s	remaining: 28m 2s
9:	test: 0.8221034	test1: 0.8103552	best: 0.8107787 (6)	total: 3.39s	remaining: 28m 11s
10:	test: 0.8228413	test1: 0.8116365	best: 0.8116365 (10)	total: 3.81s	remaining: 28m 46s
11

92:	test: 0.8405454	test1: 0.8254347	best: 0.8254347 (92)	total: 24.2s	remaining: 21m 14s
93:	test: 0.8406646	test1: 0.8255213	best: 0.8255213 (93)	total: 24.3s	remaining: 21m 9s
94:	test: 0.8407427	test1: 0.8255458	best: 0.8255458 (94)	total: 24.5s	remaining: 21m 5s
95:	test: 0.8408112	test1: 0.8255994	best: 0.8255994 (95)	total: 24.7s	remaining: 21m
96:	test: 0.8409112	test1: 0.8256858	best: 0.8256858 (96)	total: 25s	remaining: 21m 4s
97:	test: 0.8409967	test1: 0.8256401	best: 0.8256858 (96)	total: 25.2s	remaining: 20m 59s
98:	test: 0.8411053	test1: 0.8257051	best: 0.8257051 (98)	total: 25.4s	remaining: 20m 55s
99:	test: 0.8412461	test1: 0.8258233	best: 0.8258233 (99)	total: 25.5s	remaining: 20m 51s
100:	test: 0.8413385	test1: 0.8258436	best: 0.8258436 (100)	total: 25.7s	remaining: 20m 47s
101:	test: 0.8414658	test1: 0.8258966	best: 0.8258966 (101)	total: 25.9s	remaining: 20m 42s
102:	test: 0.8415887	test1: 0.8259383	best: 0.8259383 (102)	total: 26.1s	remaining: 20m 42s
103:	test: 0.

182:	test: 0.8483175	test1: 0.8289029	best: 0.8289029 (182)	total: 52.1s	remaining: 22m 51s
183:	test: 0.8483738	test1: 0.8288908	best: 0.8289029 (182)	total: 52.5s	remaining: 22m 54s
184:	test: 0.8484902	test1: 0.8289165	best: 0.8289165 (184)	total: 52.7s	remaining: 22m 52s
185:	test: 0.8485570	test1: 0.8289857	best: 0.8289857 (185)	total: 52.9s	remaining: 22m 49s
186:	test: 0.8485791	test1: 0.8289879	best: 0.8289879 (186)	total: 53.1s	remaining: 22m 46s
187:	test: 0.8486759	test1: 0.8289727	best: 0.8289879 (186)	total: 53.3s	remaining: 22m 45s
188:	test: 0.8487501	test1: 0.8290028	best: 0.8290028 (188)	total: 53.8s	remaining: 22m 48s
189:	test: 0.8488175	test1: 0.8289945	best: 0.8290028 (188)	total: 54.1s	remaining: 22m 48s
190:	test: 0.8489035	test1: 0.8289822	best: 0.8290028 (188)	total: 54.4s	remaining: 22m 49s
191:	test: 0.8489610	test1: 0.8290050	best: 0.8290050 (191)	total: 54.6s	remaining: 22m 47s
192:	test: 0.8490133	test1: 0.8290309	best: 0.8290309 (192)	total: 55.1s	remaini

272:	test: 0.8542839	test1: 0.8305324	best: 0.8305542 (269)	total: 1m 15s	remaining: 21m 50s
273:	test: 0.8543329	test1: 0.8305231	best: 0.8305542 (269)	total: 1m 16s	remaining: 21m 50s
274:	test: 0.8543957	test1: 0.8305697	best: 0.8305697 (274)	total: 1m 16s	remaining: 21m 50s
275:	test: 0.8544682	test1: 0.8305752	best: 0.8305752 (275)	total: 1m 16s	remaining: 21m 50s
276:	test: 0.8545419	test1: 0.8305652	best: 0.8305752 (275)	total: 1m 17s	remaining: 21m 53s
277:	test: 0.8545833	test1: 0.8306097	best: 0.8306097 (277)	total: 1m 17s	remaining: 21m 53s
278:	test: 0.8546416	test1: 0.8306337	best: 0.8306337 (278)	total: 1m 17s	remaining: 21m 52s
279:	test: 0.8547157	test1: 0.8306475	best: 0.8306475 (279)	total: 1m 17s	remaining: 21m 51s
280:	test: 0.8548227	test1: 0.8306720	best: 0.8306720 (280)	total: 1m 18s	remaining: 21m 52s
281:	test: 0.8548865	test1: 0.8307218	best: 0.8307218 (281)	total: 1m 18s	remaining: 21m 50s
282:	test: 0.8549622	test1: 0.8307892	best: 0.8307892 (282)	total: 1m 

361:	test: 0.8603527	test1: 0.8316885	best: 0.8317608 (360)	total: 1m 43s	remaining: 22m 7s
362:	test: 0.8604533	test1: 0.8317078	best: 0.8317608 (360)	total: 1m 44s	remaining: 22m 14s
363:	test: 0.8605342	test1: 0.8317173	best: 0.8317608 (360)	total: 1m 45s	remaining: 22m 18s
364:	test: 0.8605931	test1: 0.8317375	best: 0.8317608 (360)	total: 1m 45s	remaining: 22m 24s
365:	test: 0.8606719	test1: 0.8317458	best: 0.8317608 (360)	total: 1m 46s	remaining: 22m 27s
366:	test: 0.8607039	test1: 0.8317829	best: 0.8317829 (366)	total: 1m 46s	remaining: 22m 28s
367:	test: 0.8607453	test1: 0.8317799	best: 0.8317829 (366)	total: 1m 47s	remaining: 22m 28s
368:	test: 0.8608058	test1: 0.8317732	best: 0.8317829 (366)	total: 1m 47s	remaining: 22m 29s
369:	test: 0.8608671	test1: 0.8318115	best: 0.8318115 (369)	total: 1m 47s	remaining: 22m 29s
370:	test: 0.8609515	test1: 0.8318181	best: 0.8318181 (370)	total: 1m 48s	remaining: 22m 29s
371:	test: 0.8609606	test1: 0.8318436	best: 0.8318436 (371)	total: 1m 4

...

815:	test: 0.8862922	test1: 0.8468771	best: 0.8469922 (779)	total: 3m 4s	remaining: 15m 44s
816:	test: 0.8863402	test1: 0.8469115	best: 0.8469922 (779)	total: 3m 4s	remaining: 15m 44s
817:	test: 0.8864069	test1: 0.8468848	best: 0.8469922 (779)	total: 3m 4s	remaining: 15m 44s
818:	test: 0.8864644	test1: 0.8468812	best: 0.8469922 (779)	total: 3m 4s	remaining: 15m 43s
819:	test: 0.8864908	test1: 0.8468738	best: 0.8469922 (779)	total: 3m 5s	remaining: 15m 43s
820:	test: 0.8865358	test1: 0.8468953	best: 0.8469922 (779)	total: 3m 5s	remaining: 15m 42s
821:	test: 0.8865878	test1: 0.8468936	best: 0.8469922 (779)	total: 3m 5s	remaining: 15m 42s
822:	test: 0.8866174	test1: 0.8469124	best: 0.8469922 (779)	total: 3m 5s	remaining: 15m 42s
823:	test: 0.8866817	test1: 0.8469322	best: 0.8469922 (779)	total: 3m 5s	remaining: 15m 42s
824:	test: 0.8867480	test1: 0.8469246	best: 0.8469922 (779)	total: 3m 6s	remaining: 15m 41s
825:	test: 0.8867864	test1: 0.8469083	best: 0.8469922 (779)	total: 3m 6s	remaini

904:	test: 0.8906557	test1: 0.8468921	best: 0.8469922 (779)	total: 3m 23s	remaining: 15m 20s
905:	test: 0.8907050	test1: 0.8469326	best: 0.8469922 (779)	total: 3m 23s	remaining: 15m 20s
906:	test: 0.8907411	test1: 0.8469091	best: 0.8469922 (779)	total: 3m 23s	remaining: 15m 20s
907:	test: 0.8908069	test1: 0.8469129	best: 0.8469922 (779)	total: 3m 24s	remaining: 15m 20s
908:	test: 0.8908465	test1: 0.8469234	best: 0.8469922 (779)	total: 3m 24s	remaining: 15m 19s
909:	test: 0.8909120	test1: 0.8469700	best: 0.8469922 (779)	total: 3m 24s	remaining: 15m 19s
910:	test: 0.8909489	test1: 0.8469750	best: 0.8469922 (779)	total: 3m 24s	remaining: 15m 19s
911:	test: 0.8909984	test1: 0.8469481	best: 0.8469922 (779)	total: 3m 25s	remaining: 15m 18s
912:	test: 0.8910672	test1: 0.8469629	best: 0.8469922 (779)	total: 3m 25s	remaining: 15m 19s
913:	test: 0.8911281	test1: 0.8469253	best: 0.8469922 (779)	total: 3m 25s	remaining: 15m 18s
914:	test: 0.8911599	test1: 0.8469203	best: 0.8469922 (779)	total: 3m 

993:	test: 0.8948825	test1: 0.8470665	best: 0.8473313 (948)	total: 3m 44s	remaining: 15m 6s
994:	test: 0.8949231	test1: 0.8470546	best: 0.8473313 (948)	total: 3m 45s	remaining: 15m 6s
995:	test: 0.8949607	test1: 0.8470534	best: 0.8473313 (948)	total: 3m 45s	remaining: 15m 6s
996:	test: 0.8949906	test1: 0.8470741	best: 0.8473313 (948)	total: 3m 45s	remaining: 15m 6s
997:	test: 0.8950367	test1: 0.8470810	best: 0.8473313 (948)	total: 3m 46s	remaining: 15m 6s
998:	test: 0.8950899	test1: 0.8471038	best: 0.8473313 (948)	total: 3m 46s	remaining: 15m 6s
999:	test: 0.8951429	test1: 0.8471074	best: 0.8473313 (948)	total: 3m 46s	remaining: 15m 6s
1000:	test: 0.8951903	test1: 0.8470988	best: 0.8473313 (948)	total: 3m 47s	remaining: 15m 8s
1001:	test: 0.8952369	test1: 0.8470851	best: 0.8473313 (948)	total: 3m 47s	remaining: 15m 8s
1002:	test: 0.8952760	test1: 0.8470794	best: 0.8473313 (948)	total: 3m 47s	remaining: 15m 8s
1003:	test: 0.8953231	test1: 0.8470819	best: 0.8473313 (948)	total: 3m 48s	re

1082:	test: 0.8987951	test1: 0.8468198	best: 0.8473313 (948)	total: 4m 5s	remaining: 14m 48s
1083:	test: 0.8988304	test1: 0.8468252	best: 0.8473313 (948)	total: 4m 5s	remaining: 14m 48s
1084:	test: 0.8988611	test1: 0.8467986	best: 0.8473313 (948)	total: 4m 6s	remaining: 14m 48s
1085:	test: 0.8989082	test1: 0.8468074	best: 0.8473313 (948)	total: 4m 6s	remaining: 14m 48s
1086:	test: 0.8989444	test1: 0.8467929	best: 0.8473313 (948)	total: 4m 6s	remaining: 14m 47s
1087:	test: 0.8989731	test1: 0.8467843	best: 0.8473313 (948)	total: 4m 6s	remaining: 14m 47s
1088:	test: 0.8990251	test1: 0.8468019	best: 0.8473313 (948)	total: 4m 7s	remaining: 14m 47s
1089:	test: 0.8990948	test1: 0.8468216	best: 0.8473313 (948)	total: 4m 7s	remaining: 14m 46s
1090:	test: 0.8991482	test1: 0.8468381	best: 0.8473313 (948)	total: 4m 7s	remaining: 14m 46s
1091:	test: 0.8991845	test1: 0.8468641	best: 0.8473313 (948)	total: 4m 7s	remaining: 14m 46s
1092:	test: 0.8992078	test1: 0.8468560	best: 0.8473313 (948)	total: 4m

1171:	test: 0.9025653	test1: 0.8464782	best: 0.8473313 (948)	total: 4m 25s	remaining: 14m 27s
1172:	test: 0.9026157	test1: 0.8464240	best: 0.8473313 (948)	total: 4m 25s	remaining: 14m 27s
1173:	test: 0.9026579	test1: 0.8464142	best: 0.8473313 (948)	total: 4m 26s	remaining: 14m 26s
1174:	test: 0.9027190	test1: 0.8464235	best: 0.8473313 (948)	total: 4m 26s	remaining: 14m 26s
1175:	test: 0.9027773	test1: 0.8464238	best: 0.8473313 (948)	total: 4m 26s	remaining: 14m 26s
1176:	test: 0.9028139	test1: 0.8464552	best: 0.8473313 (948)	total: 4m 26s	remaining: 14m 25s
1177:	test: 0.9028598	test1: 0.8464569	best: 0.8473313 (948)	total: 4m 26s	remaining: 14m 25s
1178:	test: 0.9028895	test1: 0.8464395	best: 0.8473313 (948)	total: 4m 27s	remaining: 14m 25s
1179:	test: 0.9029246	test1: 0.8464345	best: 0.8473313 (948)	total: 4m 27s	remaining: 14m 25s
1180:	test: 0.9029767	test1: 0.8464047	best: 0.8473313 (948)	total: 4m 27s	remaining: 14m 25s
1181:	test: 0.9030232	test1: 0.8464225	best: 0.8473313 (948)

In [34]:
y_pred_1=np.mean(totcb_pred,0)
y_pred_1

array([0.31176965, 0.37773693, 0.38980719, ..., 0.25829964, 0.51273116,
       0.19005462])

In [35]:
cat_sub = SampleSub.copy()

In [36]:
cat_sub['default_status']=y_pred_1

In [37]:
cat_sub.to_csv('CAT_SUB.csv',index=False)

## Model 3- LightGBM Classifier

10 Stratified KFold LightGBM training and prediction on test dataset.

In [38]:
totlgb_pred = []

fold = StratifiedKFold(n_splits=10, shuffle=False, random_state=42)

for train_index,test_index in fold.split(x,y):
    x_train,x_test=x[train_index], x[test_index]
    y_train,y_test=y[train_index], y[test_index]
    
    estimator_2=lgb.LGBMClassifier(learning_rate=0.150, metric='l1', 
                                early_stopping_rounds =200, 
                                eval_metric ='roc_auc_score',max_depth = -1,
                                min_child_samples=100,n_estimators =100)
    
    estimator_2.fit(x_train,y_train, eval_set=[(x_train,y_train), (x_test,y_test)])
    
    prediction_2 = estimator_2.predict_proba(x_test)[:,1] 

    print("auc_score: ",roc_auc_score(y_test, prediction_2))
    
    pred2=estimator_2.predict_proba(test)[:,1]
    
    totlgb_pred.append(pred2)

[1]	training's l1: 0.356897	valid_1's l1: 0.357839
Training until validation scores don't improve for 200 rounds
[2]	training's l1: 0.345672	valid_1's l1: 0.347704
[3]	training's l1: 0.335908	valid_1's l1: 0.339037
[4]	training's l1: 0.327385	valid_1's l1: 0.331006
[5]	training's l1: 0.319821	valid_1's l1: 0.324202
[6]	training's l1: 0.313395	valid_1's l1: 0.318462
[7]	training's l1: 0.307522	valid_1's l1: 0.313086
[8]	training's l1: 0.302275	valid_1's l1: 0.308355
[9]	training's l1: 0.297691	valid_1's l1: 0.304015
[10]	training's l1: 0.293715	valid_1's l1: 0.300546
[11]	training's l1: 0.289943	valid_1's l1: 0.297172
[12]	training's l1: 0.286743	valid_1's l1: 0.294449
[13]	training's l1: 0.283705	valid_1's l1: 0.291909
[14]	training's l1: 0.281015	valid_1's l1: 0.28959
[15]	training's l1: 0.2785	valid_1's l1: 0.287191
[16]	training's l1: 0.276329	valid_1's l1: 0.285274
[17]	training's l1: 0.274214	valid_1's l1: 0.283709
[18]	training's l1: 0.27244	valid_1's l1: 0.282253
[19]	training's

[55]	training's l1: 0.250605	valid_1's l1: 0.263507
[56]	training's l1: 0.250375	valid_1's l1: 0.263484
[57]	training's l1: 0.25001	valid_1's l1: 0.263419
[58]	training's l1: 0.24979	valid_1's l1: 0.263396
[59]	training's l1: 0.249426	valid_1's l1: 0.263306
[60]	training's l1: 0.249102	valid_1's l1: 0.263217
[61]	training's l1: 0.248895	valid_1's l1: 0.263139
[62]	training's l1: 0.248611	valid_1's l1: 0.263098
[63]	training's l1: 0.248402	valid_1's l1: 0.26313
[64]	training's l1: 0.248094	valid_1's l1: 0.263143
[65]	training's l1: 0.247758	valid_1's l1: 0.262977
[66]	training's l1: 0.247466	valid_1's l1: 0.262919
[67]	training's l1: 0.247131	valid_1's l1: 0.262913
[68]	training's l1: 0.246859	valid_1's l1: 0.262791
[69]	training's l1: 0.24657	valid_1's l1: 0.26268
[70]	training's l1: 0.246406	valid_1's l1: 0.262726
[71]	training's l1: 0.24626	valid_1's l1: 0.262799
[72]	training's l1: 0.245948	valid_1's l1: 0.262691
[73]	training's l1: 0.245671	valid_1's l1: 0.262564
[74]	training's l1

[7]	training's l1: 0.30763	valid_1's l1: 0.315333
[8]	training's l1: 0.30228	valid_1's l1: 0.310785
[9]	training's l1: 0.297703	valid_1's l1: 0.306962
[10]	training's l1: 0.293567	valid_1's l1: 0.303261
[11]	training's l1: 0.28979	valid_1's l1: 0.299995
[12]	training's l1: 0.286583	valid_1's l1: 0.297215
[13]	training's l1: 0.283596	valid_1's l1: 0.294753
[14]	training's l1: 0.280857	valid_1's l1: 0.292457
[15]	training's l1: 0.278439	valid_1's l1: 0.29055
[16]	training's l1: 0.276423	valid_1's l1: 0.28889
[17]	training's l1: 0.27433	valid_1's l1: 0.287111
[18]	training's l1: 0.27257	valid_1's l1: 0.285692
[19]	training's l1: 0.27098	valid_1's l1: 0.28445
[20]	training's l1: 0.269581	valid_1's l1: 0.283289
[21]	training's l1: 0.268108	valid_1's l1: 0.282241
[22]	training's l1: 0.266905	valid_1's l1: 0.281333
[23]	training's l1: 0.2657	valid_1's l1: 0.280455
[24]	training's l1: 0.264663	valid_1's l1: 0.27964
[25]	training's l1: 0.263648	valid_1's l1: 0.278823
[26]	training's l1: 0.26258

[63]	training's l1: 0.247343	valid_1's l1: 0.263574
[64]	training's l1: 0.247026	valid_1's l1: 0.26356
[65]	training's l1: 0.246841	valid_1's l1: 0.263566
[66]	training's l1: 0.246571	valid_1's l1: 0.263499
[67]	training's l1: 0.246225	valid_1's l1: 0.263398
[68]	training's l1: 0.245837	valid_1's l1: 0.2633
[69]	training's l1: 0.24548	valid_1's l1: 0.263146
[70]	training's l1: 0.245218	valid_1's l1: 0.263174
[71]	training's l1: 0.24499	valid_1's l1: 0.263154
[72]	training's l1: 0.24471	valid_1's l1: 0.263023
[73]	training's l1: 0.244472	valid_1's l1: 0.262921
[74]	training's l1: 0.244122	valid_1's l1: 0.262826
[75]	training's l1: 0.243831	valid_1's l1: 0.262791
[76]	training's l1: 0.243503	valid_1's l1: 0.262729
[77]	training's l1: 0.243318	valid_1's l1: 0.262756
[78]	training's l1: 0.243171	valid_1's l1: 0.26274
[79]	training's l1: 0.242896	valid_1's l1: 0.262635
[80]	training's l1: 0.24269	valid_1's l1: 0.262733
[81]	training's l1: 0.242421	valid_1's l1: 0.26272
[82]	training's l1: 0

[15]	training's l1: 0.278056	valid_1's l1: 0.293373
[16]	training's l1: 0.27581	valid_1's l1: 0.291535
[17]	training's l1: 0.273782	valid_1's l1: 0.289978
[18]	training's l1: 0.271976	valid_1's l1: 0.288626
[19]	training's l1: 0.270395	valid_1's l1: 0.287415
[20]	training's l1: 0.268835	valid_1's l1: 0.286304
[21]	training's l1: 0.267379	valid_1's l1: 0.285207
[22]	training's l1: 0.265931	valid_1's l1: 0.28406
[23]	training's l1: 0.264773	valid_1's l1: 0.283228
[24]	training's l1: 0.263753	valid_1's l1: 0.282614
[25]	training's l1: 0.262769	valid_1's l1: 0.281907
[26]	training's l1: 0.261829	valid_1's l1: 0.281252
[27]	training's l1: 0.261048	valid_1's l1: 0.280829
[28]	training's l1: 0.260184	valid_1's l1: 0.280253
[29]	training's l1: 0.259552	valid_1's l1: 0.279891
[30]	training's l1: 0.258758	valid_1's l1: 0.279411
[31]	training's l1: 0.257979	valid_1's l1: 0.278947
[32]	training's l1: 0.257263	valid_1's l1: 0.278489
[33]	training's l1: 0.256837	valid_1's l1: 0.278258
[34]	training'

[73]	training's l1: 0.244824	valid_1's l1: 0.264426
[74]	training's l1: 0.244661	valid_1's l1: 0.264383
[75]	training's l1: 0.244457	valid_1's l1: 0.264361
[76]	training's l1: 0.244117	valid_1's l1: 0.264311
[77]	training's l1: 0.243884	valid_1's l1: 0.264233
[78]	training's l1: 0.243602	valid_1's l1: 0.264107
[79]	training's l1: 0.243305	valid_1's l1: 0.26408
[80]	training's l1: 0.243091	valid_1's l1: 0.264017
[81]	training's l1: 0.242871	valid_1's l1: 0.263896
[82]	training's l1: 0.242617	valid_1's l1: 0.263897
[83]	training's l1: 0.242345	valid_1's l1: 0.263833
[84]	training's l1: 0.242053	valid_1's l1: 0.263752
[85]	training's l1: 0.241732	valid_1's l1: 0.263631
[86]	training's l1: 0.241524	valid_1's l1: 0.26363
[87]	training's l1: 0.241341	valid_1's l1: 0.263578
[88]	training's l1: 0.241151	valid_1's l1: 0.263595
[89]	training's l1: 0.240899	valid_1's l1: 0.263566
[90]	training's l1: 0.240707	valid_1's l1: 0.26357
[91]	training's l1: 0.24043	valid_1's l1: 0.263445
[92]	training's 

[24]	training's l1: 0.265512	valid_1's l1: 0.269471
[25]	training's l1: 0.264567	valid_1's l1: 0.268738
[26]	training's l1: 0.263715	valid_1's l1: 0.268122
[27]	training's l1: 0.262855	valid_1's l1: 0.267568
[28]	training's l1: 0.262161	valid_1's l1: 0.267205
[29]	training's l1: 0.261437	valid_1's l1: 0.266642
[30]	training's l1: 0.260797	valid_1's l1: 0.266205
[31]	training's l1: 0.260103	valid_1's l1: 0.265879
[32]	training's l1: 0.259461	valid_1's l1: 0.265318
[33]	training's l1: 0.258841	valid_1's l1: 0.264982
[34]	training's l1: 0.258387	valid_1's l1: 0.264761
[35]	training's l1: 0.257815	valid_1's l1: 0.264453
[36]	training's l1: 0.257277	valid_1's l1: 0.264167
[37]	training's l1: 0.256705	valid_1's l1: 0.26391
[38]	training's l1: 0.256279	valid_1's l1: 0.263661
[39]	training's l1: 0.255736	valid_1's l1: 0.263375
[40]	training's l1: 0.255365	valid_1's l1: 0.263223
[41]	training's l1: 0.254977	valid_1's l1: 0.262962
[42]	training's l1: 0.254532	valid_1's l1: 0.262692
[43]	training

In [39]:
y_pred_2=np.mean(totlgb_pred,0)
y_pred_2

array([0.31658459, 0.33067228, 0.4038121 , ..., 0.23126225, 0.56874707,
       0.19125135])

In [40]:
lgb_sub = SampleSub.copy()

In [41]:
lgb_sub['default_status']=y_pred_2

In [42]:
lgb_sub.to_csv('LGB_SUB.csv',index=False)

## Blending of predictions 

Level 2 blending

In [106]:
## BLEND
blend = ((rfc_sub['default_status']*0.25) + (lgb_sub['default_status']*0.50) +(cat_sub['default_status']*0.80))*0.95 + (cat_sub['default_status']*0.75)

## BLEND1
blend1 = ((rfc_sub['default_status']*0.25) + (lgb_sub['default_status']*0.50) +(cat_sub['default_status']*0.75))*0.95 + (cat_sub['default_status']*0.5)

Level 3 blending

In [101]:
final_blend = (blend1*0.5) + (blend*0.3)+(rfc_sub['default_status']*0.1)

## Submission

In [103]:
Sample=pd.read_csv('SampleSubmission.csv')
blend_sub=Sample.copy()

In [104]:
blend_sub['default_status']=final_blend

In [105]:
blend_sub.to_csv('BLEND1.csv',index=False)